### Задание 2
**Цели:**
1. Подготовить данные и целевую переменную
2. Разбить данные на train/test
3. Посчитать константные предсказания
4. Создать простой бейзлайн
5. Оценить качество на выбранной метрике
6. Добиться воспроизводимости

In [16]:
!pip -q install kagglehub


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


Импорты необходимых библиотек и фиксирование seed

In [18]:
import os
import random
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

import kagglehub

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

Загрузка набора данных о ценнах на недвижимость в Мумбаи с сайта https://www.kaggle.com ,проверка успешности скачивания

In [20]:
DATASET_NAME = "kevinnadar22/mumbai-house-price-data-70k-entries"

data_dir = kagglehub.dataset_download(DATASET_NAME)
csv_path = next(
    (os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(".csv")),
    None,
)

if csv_path is None:
    raise FileNotFoundError("CSV не найден!")

df = pd.read_csv(csv_path)

Предобработка данных. Удаление дубликатов и логарифмирование целевой переменной. Это преобразование уменьшает влияние экстремально дорогих объектов и стабилизирует дисперсию - это помогает простым моделям.

In [22]:
df = df.drop_duplicates()
df["log_price"] = np.log1p(df["price"])

target = "log_price"
features = [
    "area",
    "bedroom_num",
    "bathroom_num",
    "balcony_num",
    "age",
    "total_floors",
    "property_type",
    "furnished",
    "locality",
]

X = df[features]
y = df[target]

Разбиваем на 80/20 набор данных и фиксируем random_state, смотрим размерность разбивки.

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)


X_train.shape, X_test.shape, y_train.shape, y_test.shape

((41413, 9), (10354, 9), (41413,), (10354,))

Проверяем воспроизводимость разбивки данных.

In [28]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(
    X, y, test_size=0.2, random_state=42
)

assert (X_train.index == X_train2.index).all()
assert (X_test.index == X_test2.index).all()
print("Данные воспроизводимы")

Данные воспроизводимы


### Константные предсказания

1. Расчёт **среднего** и **медианы** логарифма цены по обучающей выборке.  
2. Проверка качества этих прогнозов на тестовой выборке с помощью **RMSE**.  
3. **RMSE** выбрана как основная метрика, поскольку она сильнее штрафует большие ошибки — это критично при прогнозировании цен на дорогую недвижимость.  
4. **Медиана** добавлена для оценки устойчивости к выбросам.  
5. Дополнительно вычислены **средняя** и **медианная цены в рупиях** для бизнес-интерпретации результатов.

In [32]:
mean_value = float(np.mean(y_train))
median_value = float(np.median(y_train))

y_pred_mean = np.full(shape=y_test.shape, fill_value=mean_value, dtype=float)
y_pred_median = np.full(shape=y_test.shape, fill_value=median_value, dtype=float)

rmse_mean = mean_squared_error(y_test, y_pred_mean) ** 0.5
rmse_median = mean_squared_error(y_test, y_pred_median) ** 0.5

mean_price_inr = float(np.expm1(mean_value))
median_price_inr = float(np.expm1(median_value))

print(f"Базовая модель (среднее): RMSE = {rmse_mean:.4f}")
print(f"Базовая модель (медиана): RMSE = {rmse_median:.4f}")
print(f"Средняя цена в выборке: {mean_price_inr:,.0f} ₨")
print(f"Медианная цена в выборке: {median_price_inr:,.0f} ₨")

Базовая модель (среднее): RMSE = 0.9203
Базовая модель (медиана): RMSE = 0.9207
Средняя цена в выборке: 12,465,149 ₨
Медианная цена в выборке: 12,000,339 ₨


### Константные предсказания

1. Константные модели дают одинаковые предсказания — одна и та же цена для всех объектов.  
2. **RMSE = 0.92** показывает, что ошибка такой модели довольно велика: она не учитывает различия между квартирами (площадь, расположение, тип недвижимости и т.д.).  
3. **Средняя и медианная цены** (около 12 млн рупий) отражают общий уровень стоимости жилья в выборке.  
4. Подобные предсказания полезны как **точка отсчёта**, которую должна превзойти любая осмысленная модель.

### Бейзлайновая модель: линейная регрессия

1. Выбрана **линейная регрессия** из-за её простоты и интерпретируемости при прогнозировании логарифма цены. Это классическое решение для задач оценки стоимости недвижимости.  
2. **Числовые признаки** (площадь, количество комнат и т.д.) передаются напрямую.  
3. Для **категориальных признаков** используется `OneHotEncoder`, который превращает каждое значение в отдельный бинарный признак (0/1) и не создаёт ложных числовых отношений между категориями. Это важно для линейной модели, которая предполагает числовой, но не упорядоченный ввод.  
4. `ColumnTransformer` обрабатывает числовые и категориальные признаки раздельно, а `Pipeline` объединяет предобработку и обучение в единый процесс. Это повышает воспроизводимость и удобство повторного запуска.  
5. В ходе обучения модель подбирает коэффициенты, минимизируя ошибку между предсказанными и фактическими логарифмами цен.  
6. После обучения вычисляются метрики **RMSE**, **MAE** и **R²** для оценки точности прогноза.  
7. Затем логарифмические предсказания переводятся обратно в реальные рупии с помощью `np.expm1`, что позволяет оценить ошибки в привычных денежных единицах. Это помогает понять, насколько модель ошибается в абсолютных значениях цены.

In [34]:
numeric_features = [
    "area",
    "bedroom_num",
    "bathroom_num",
    "balcony_num",
    "age",
    "total_floors",
]
categorical_features = ["property_type", "furnished", "locality"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)

model = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("regressor", LinearRegression()),
    ]
)

model.fit(X_train, y_train)

y_pred_log = model.predict(X_test)
rmse_log = mean_squared_error(y_test, y_pred_log) ** 0.5
mae_log = mean_absolute_error(y_test, y_pred_log)
r2 = r2_score(y_test, y_pred_log)

y_test_inr = np.expm1(y_test)
y_pred_inr = np.expm1(y_pred_log)
rmse_inr = mean_squared_error(y_test_inr, y_pred_inr) ** 0.5
mae_inr = mean_absolute_error(y_test_inr, y_pred_inr)

print(f"Ошибка RMSE (в логарифмах): {rmse_log:.4f}")
print(f"Средняя абсолютная ошибка MAE (в логарифмах): {mae_log:.4f}")
print(f"Коэффициент детерминации R²: {r2:.4f}")
print(f"Ошибка RMSE (в рупиях): {rmse_inr:,.0f} ₨")
print(f"Средняя абсолютная ошибка MAE (в рупиях): {mae_inr:,.0f} ₨")

Ошибка RMSE (в логарифмах): 0.3087
Средняя абсолютная ошибка MAE (в логарифмах): 0.2217
Коэффициент детерминации R²: 0.8875
Ошибка RMSE (в рупиях): 25,211,910 ₨
Средняя абсолютная ошибка MAE (в рупиях): 5,390,856 ₨


### Результаты

1. **Линейная регрессия** показала высокое качество предсказаний: значение **R² = 0.8875** означает, что модель объясняет около **89 % вариации цен недвижимости** в Мумбае.  
2. **Средняя абсолютная ошибка (MAE)** составляет примерно **5.4 млн ₨**, а **среднеквадратичная ошибка (RMSE)** — около **25.2 млн ₨**, что указывает на наличие нескольких очень дорогих объектов, значительно влияющих на RMSE.  
3. В целом, модель **адекватно оценивает цены большинства объектов** и может служить **надёжным бейзлайном** для дальнейшего улучшения — например, с помощью регуляризации или нелинейных моделей.